In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [4]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: superuser_9 (scam_sharks). Use `wandb login --relogin` to force relogin


True

In [5]:
df = pd.read_csv('../../data/credit_filtered.csv')

X = df[['type','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']]
y = df['isFraud']

le = LabelEncoder()
X['type_encoded'] = le.fit_transform(X['type'])

X = X.drop(['type'],axis=1)

/var/folders/38/djv715md7dz__89srf2wkcsh0000gn/T/ipykernel_1061/923524054.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['type_encoded'] = le.fit_transform(X['type'])


In [6]:
from sklearn.model_selection import cross_val_score, train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [11]:
# Train model, get predictions
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_probas = model.predict_proba(X_test)
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

In [7]:
wandb.init(project="credit_fraud_detect")

In [10]:
labels = ['not_fraud','fraud']
wandb.sklearn.plot_class_proportions(y_train, y_test, labels)

In [13]:
wandb.sklearn.plot_roc(y_test, y_probas, labels)

In [18]:
wandb.sklearn.plot_confusion_matrix(y_test, y_pred, labels)

In [22]:
import pickle, os

In [27]:
with open('models/model.pkl','wb') as f:
    pickle.dump(model,f)

In [28]:
art = wandb.Artifact("my-object-detector", type="model")
art.add_file("models/model.pkl")
wandb.log_artifact(art)

In [24]:
from wandb.beta.workflows import log_model
log_model(model, "Random_forest", aliases=["best"])